In [3]:
#!/usr/bin/env python3
"""
IoT Intrusion Detection CNN Training Script
Author: abhiii
Dataset: /content/server_df.csv
"""

import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# -------------------- CONFIG --------------------
CSV_PATH = "/content/server_df.csv"     # ✅ dataset path
TARGET = "label"                         # column name for labels
EPOCHS = 20
BATCH_SIZE = 128
MODEL_SAVE_PATH = "iot_cnn_model.h5"
BEST_MODEL_PATH = "iot_cnn_best.h5"
SCALER_PATH = "iot_scaler.joblib"
ENCODER_PATH = "iot_label_encoder.joblib"
REPORT_PATH = "iot_evaluation_report.csv"
# ------------------------------------------------

# 1️⃣ Load dataset
df = pd.read_csv(CSV_PATH)
print("✅ Dataset loaded:", df.shape)
print("Columns:", df.columns.tolist())

# 2️⃣ Separate features and labels
y = df[TARGET].copy()
X = df.drop(columns=[TARGET])

# 3️⃣ Encode categorical features if present
cat_cols = X.select_dtypes(include=['object','category']).columns.tolist()
if cat_cols:
    print("ℹ️ Encoding categorical columns:", cat_cols)
    X = pd.get_dummies(X, columns=cat_cols, drop_first=True)

# Encode target labels if categorical
if y.dtype == object or y.dtype.name == 'category':
    le = LabelEncoder()
    y = le.fit_transform(y)
    joblib.dump(le, ENCODER_PATH)
    print("✅ Label encoder saved:", ENCODER_PATH)

# 4️⃣ Remove classes with < 2 samples to avoid stratify error
df_full = pd.concat([pd.DataFrame(X), pd.Series(y, name="label")], axis=1)
df_full = df_full.groupby("label").filter(lambda x: len(x) > 1)
X = df_full.drop("label", axis=1)
y = df_full["label"]
print("✅ Filtered dataset shape (after removing rare classes):", X.shape)

# 5️⃣ Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.values)
joblib.dump(scaler, SCALER_PATH)
print("✅ Scaler saved:", SCALER_PATH)

# 6️⃣ Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)
print("✅ Train/Test split done:", X_train.shape, X_test.shape)

# 7️⃣ Reshape for Conv1D
n_features = X_train.shape[1]
X_train_cnn = X_train.reshape((-1, n_features, 1))
X_test_cnn = X_test.reshape((-1, n_features, 1))
print("📐 CNN input shape:", X_train_cnn.shape)

# 8️⃣ Build CNN model
def build_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(64, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Conv1D(128, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        GlobalMaxPooling1D(),
        Dense(64, activation='relu'),
        Dropout(0.4),
        Dense(1, activation='sigmoid') if num_classes == 2 else Dense(num_classes, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy' if num_classes == 2 else 'sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

num_classes = len(np.unique(y))
model = build_model((n_features, 1), num_classes)
model.summary()

# 9️⃣ Train the model
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint(BEST_MODEL_PATH, save_best_only=True, monitor='val_loss')
]

history = model.fit(
    X_train_cnn, y_train,
    validation_split=0.15,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=2
)

# 🔟 Save trained model
model.save(MODEL_SAVE_PATH)
print("✅ Model saved:", MODEL_SAVE_PATH)
print("✅ Best checkpoint saved:", BEST_MODEL_PATH)

# 1️⃣1️⃣ Evaluate model
y_pred_prob = model.predict(X_test_cnn)
if num_classes == 2:
    y_pred = (y_pred_prob.flatten() > 0.5).astype(int)
else:
    y_pred = np.argmax(y_pred_prob, axis=1)

report = classification_report(y_test, y_pred, output_dict=True)
pd.DataFrame(report).transpose().to_csv(REPORT_PATH)
print("📊 Evaluation report saved:", REPORT_PATH)

print("\n🎉 Training Complete! Your IoT CNN Intrusion Detection Model is ready 🚀")


✅ Dataset loaded: (446949, 47)
Columns: ['flow_duration', 'Header_Length', 'Protocol Type', 'Duration', 'Rate', 'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number', 'rst_flag_number', 'psh_flag_number', 'ack_flag_number', 'ece_flag_number', 'cwr_flag_number', 'ack_count', 'syn_count', 'fin_count', 'urg_count', 'rst_count', 'HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC', 'Tot sum', 'Min', 'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number', 'Magnitue', 'Radius', 'Covariance', 'Variance', 'Weight', 'label']
✅ Label encoder saved: iot_label_encoder.joblib
✅ Filtered dataset shape (after removing rare classes): (446948, 46)
✅ Scaler saved: iot_scaler.joblib
✅ Train/Test split done: (357558, 46) (89390, 46)
📐 CNN input shape: (357558, 46, 1)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 46, 64)         │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 46, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 23, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 23, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 23, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 34)             │         2,210 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 36,194 (141.38 KB)

 Trainable params: 35,810 (139.88 KB)

 Non-trainable params: 384 (1.50 KB)

Epoch 1/20


2375/2375 - 84s - 36ms/step - accuracy: 0.7629 - loss: 0.5948 - val_accuracy: 0.7953 - val_loss: 0.4765
Epoch 2/20
2375/2375 - 79s - 33ms/step - accuracy: 0.7902 - loss: 0.4799 - val_accuracy: 0.7969 - val_loss: 0.4776
Epoch 3/20
2375/2375 - 78s - 33ms/step - accuracy: 0.8276 - loss: 0.4053 - val_accuracy: 0.5919 - val_loss: 1.2740
Epoch 4/20
2375/2375 - 83s - 35ms/step - accuracy: 0.9142 - loss: 0.2370 - val_accuracy: 0.5945 - val_loss: 3.2156
Epoch 5/20
2375/2375 - 77s - 32ms/step - accuracy: 0.9363 - loss: 0.1776 - val_accuracy: 0.5863 - val_loss: 2.7748
Epoch 6/20
2375/2375 - 76s - 32ms/step - accuracy: 0.9186 - loss: 0.2169 - val_accuracy: 0.5947 - val_loss: 2.9089


✅ Model saved: iot_cnn_model.h5
✅ Best checkpoint saved: iot_cnn_best.h5
2794/2794 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step
📊 Evaluation report saved: iot_evaluation_report.csv

🎉 Training Complete! Your IoT CNN Intrusion Detection Model is ready 🚀


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
